# 応用情報工学演習（谷口研）深層学習ベースの画像認識 深層学習の基本 - Google Colab -

今回から応用的なPyTorchの使い方について確認していきます。GPUを利用しないと時間がかかるため、PyTorch 1回目のドキュメントを参照して、GPUランタイムを使うようにしてください。

## MNISTデータ

Modified National Institute of Standards and Technology database (MNIST) と呼ばれるデータセットを利用した数字認識をニューラルネットワークによって行います。

MNISTは、様々なパターンの手書き数字が、訓練用に6万枚、検証用に1万枚用意されています。

<img src="https://drive.google.com/uc?export=view&id=1twT3PKPpfyZ7Oi3D1jxA9BBd8YmW13_L">

機械学習・ディープラーニングプログラミングでよく利用される、代表的なデータセットです。

ディープラーニングを使わずSupport Vector Machine (SVM) などの従来型の機械学習モデルでも97~98%の精度であることが良く知られており、
あたらしく作成したモデルが著しく低い精度を示す場合、何かおかしいといったことに気がつくためのデバッグ的利用が多いです。

近年では、簡単過ぎるため、実際の精度評価には後述のCIFAR-10など、別のデータセットが使用されます。

***ライブラリのインポート***

In [ ]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from IPython.display import display

In [ ]:
# torch関連ライブラリのインポート

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [ ]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

## GPU利用

Google Colab上ではGPUが利用できます。

PyTorchでは、データがCPU、GPUのどちらにあるかを常に意識する必要があります。

GPUを利用する場合のプログラミング手法を確認します。

### GPUチェック

cuda:0と表示されれば成功。

In [ ]:
# デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

### GPU利用のルール

PyTorchでGPUを利用する場合のルールは次の通りです。

* テンソル変数はデータがCPU／GPU上のどちらにあるかを属性として持っている。
* CPUとGPU間でデータはto関数で転送する
* 2つの変数が両方ともGPU上にある場合、演算はGPUで行われる
* 変数の片方がCPU、もう一方がGPUの場合、演算はエラーになる

以下のサンプルでこのことを試してみましょう。

In [ ]:
# テスト用tensor変数x , y
x_np = np.arange(-2.0, 2.1, 0.25)
y_np = np.arange(-1.0, 3.1, 0.25)
x = torch.tensor(x_np).float()
y = torch.tensor(y_np).float()

# xとyの間の演算
z = x * y
print(z)

In [ ]:
# 変数xをGPUに送る
x = x.to(device)

# 変数xとyの属性data, deviceの確認
print('x: ', x.device)
print('y: ', y.device)

In [ ]:
# この状態でxとyの演算をすると。。。

try :
  z = x * y
except Exception as e:
  print("ERROR:", e)

In [ ]:
# yもGPUに送る
y = y.to(device)

# 今度は計算可能になる
z = x * y
print(z)

## データ前処理

学習データをモデルへ入力する前に加工することをデータ前処理といいます。

PyTorchではtorchvision.transformsというライブラリに、前処理に便利な部品がそろっていて、この部品を組み合わせて使うだけで、簡単に望む形式へのデータ変換できます。

機械学習ではデータの収集も重要な準備です。こちらも便利な仕組みが用意されています（後述）。

PyTorchで用意されているデータを利用して、まずはデータ前処理の方法を確認します。


### (前処理1) Datasetによるデータ読み込み

Datasetというクラスを利用することで、[様々なデータ](https://pytorch.org/vision/stable/datasets.html)を取得することができます。


In [ ]:
# ライブラリインポート
import torchvision.datasets as datasets

# ダウンロード先ディレクトリ名
data_root = './data'

train_set0 = datasets.MNIST(
    # 元データダウンロード先の指定
    root = data_root,
    # 訓練データか検証データか
    train = True,
    # 元データがない場合にダウンロードするか
    download = True)

In [ ]:
# ダウンロードしたファイルの確認

!ls -lR ./data/MNIST

In [ ]:
# データ件数の確認
print('データ件数: ', len(train_set0))

# 最初の要素の取得
image, label = train_set0[0]

# データ型の確認
print('入力データの型: ', type(image))
print('正解データの型: ', type(label))

In [ ]:
# 入力データの画像表示

plt.figure(figsize=(2,3))
plt.title(f'{label}')
plt.imshow(image, cmap='gray_r')
plt.axis('off')
plt.show()

In [ ]:
# 正解データ付きで、最初の20個をイメージ表示

plt.figure(figsize=(10, 3))
for i in range(20):
    ax = plt.subplot(2, 10, i + 1)

    # image と labelの取得
    image, label = train_set0[i]

    # イメージ表示
    plt.imshow(image, cmap='gray_r')
    ax.set_title(f'{label}')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

### (前処理2) Transformsによるデータ前処理

**Step1 ToTensorの利用**

In [ ]:
# ライブラリインポート
import torchvision.transforms as transforms

transform1 = transforms.Compose([
    # データのTensor化
    transforms.ToTensor(),
])

train_set1 = datasets.MNIST(
    root=data_root,  train=True,  download=True,
    transform = transform1)

In [ ]:
# 変換結果の確認

image, label = train_set1[0]
print('入力データの型: ', type(image))
print('入力データのshape: ', image.shape)
print('最小値: ', image.data.min())
print('最大値: ', image.data.max())

**Step2 Normalizeの利用**

In [ ]:
transform2 = transforms.Compose([
    # データのTensor化
    transforms.ToTensor(),

    # データの正規化
    transforms.Normalize(0.5,  0.5),
])

train_set2 = datasets.MNIST(
    root = data_root,  train = True,  download = True,
    transform = transform2)

In [ ]:
# 変換結果の確認

image, label = train_set2[0]
print('shape: ', image.shape)
print('最小値: ', image.data.min())
print('最大値: ', image.data.max())

**Step3 Lambdaを利用して1階テンソル化**

In [ ]:
transform3 = transforms.Compose([
    # データのTensor化
    transforms.ToTensor(),

    # データの正規化
    transforms.Normalize(0.5, 0.5),

    # Tensorの1階テンソル化
    transforms.Lambda(lambda x: x.view(-1)),
])

train_set3 = datasets.MNIST(
    root = data_root,  train = True,
    download=True, transform = transform3)

In [ ]:
# 変換結果の確認

image, label = train_set3[0]
print('shape: ', image.shape)
print('最小値: ', image.data.min())
print('最大値: ', image.data.max())

**最終的な実装**

In [ ]:
# データ変換用関数 Transforms
# (1) Imageをテンソル化
# (2) [0, 1]の範囲の値を[-1, 1]の範囲にする
# (3) データのshapeを[1, 28, 28]から[784]に変換

transform = transforms.Compose([
    # (1) データのテンソル化
    transforms.ToTensor(),

    # (2) データの正規化
    transforms.Normalize(0.5, 0.5),

    # (3) 1階テンソルに変換
    transforms.Lambda(lambda x: x.view(-1)),
])

In [ ]:
# データ取得用関数 Dataset

# 訓練用データセットの定義
train_set = datasets.MNIST(
    root = data_root, train = True,
    download = True, transform = transform)

# 検証データセットの定義
test_set = datasets.MNIST(
    root = data_root, train = False,
    download = True, transform = transform)

### (前処理3) データローダーによるミニバッチ用データ生成

勾配降下法で学習する場合、何件のデータで勾配計算をするかという問題があります。

一般的に学習データは多ければ多いほどよい学習結果が得られます。学習データが数万あるというのも珍しくありません。
実際、今回利用するMNISTも6万件のデータがあります。

一つずつ勾配を計算すると時間がかかるため、いくらかまとめて計算することを考えます。

事前に決めた数でグループを作り、このグループ単位で勾配計算をする方法を`ミニバッチ学習法`と呼びます。

<img src="https://drive.google.com/uc?export=view&id=1CC6E0a5fHIs-n4Av7OTMO1uMc6mKhnJL">

In [ ]:
# ライブラリインポート
from torch.utils.data import DataLoader

# ミニバッチのサイズ指定
batch_size = 500

# 訓練用データローダー
# 訓練用なので、シャッフルをかける
train_loader = DataLoader(
    train_set, batch_size = batch_size,
    shuffle = True)

# 検証用データローダー
# 検証時にシャッフルは不要
test_loader = DataLoader(
    test_set,  batch_size = batch_size,
    shuffle = False)

In [ ]:
# 何組のデータが取得できるか
print(len(train_loader))

# DataLoaderから最初の1セットを取得する
for images, labels in train_loader:
    break

print(images.shape)
print(labels.shape)

In [ ]:
# イメージ表示
plt.figure(figsize=(10, 3))
for i in range(20):
    ax = plt.subplot(2, 10, i + 1)

    # numpyに変換
    image = images[i].numpy()
    label = labels[i]

    # imgの範囲を[0, 1]に戻す
    image2 = (image + 1)/ 2
    # イメージ表示
    plt.imshow(image2.reshape(28, 28),cmap='gray_r')
    ax.set_title(f'{label}')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

全体で60,000件ある訓練データが120個のグループに分割されて取得できていることがわかります。

今回はテスト用でシャッフルなしでデータを取得しましたが、訓練データは取得のたびにシャッフルがかかります。

つまり、ミニバッチ用のデータセットが自動的に取得できていることになります。

## ニューラルネットワーク

線形層、あるいは全結合層を積み重ねたモデルをニューラルネットワーク（多層パーセプトロン）と呼び、層の数が多いものを特に深層学習（Deep Learning）と呼びます。

ヒトの神経細胞（ニューロン）ネットワークの仕組みを模しているため、人工知能（AI：Artificial Intelligence）と呼称されることもあります。

<img src="https://drive.google.com/uc?export=view&id=1mLX035DoahddOQbmEHRq462w2ug7OZvz">

### 活性化関数

ニューラルネットワークで重要な計算に`活性化関数`というものがあります。

これまでに扱った線形層をいくら重ねても線形関数にしかならないことが知られています。

活性化関数は、階層の深いニューラルネットワークを意味あるものにするために、線形関数の計算結果に対して非線形作用を与えます。

以下ではよく利用されるReLU関数を紹介しています。

**ReLU関数**


In [ ]:
# ReLU関数のグラフ

relu = nn.ReLU()
x_np = np.arange(-2, 2.1, 0.25)
x = torch.tensor(x_np).float()
y = relu(x)

plt.plot(x.data, y.data)
plt.title('ReLU')
plt.show()

### ニューラルネットワークの実装

In [ ]:
# 入力次元数
n_input = image.shape[0]

# 出力次元数
# 分類先クラス数　今回は10になる
n_output = len(set(list(labels.data.numpy())))

#   隠れ層のノード数
n_hidden = 128

# 結果確認
print(f'n_input: {n_input}  n_hidden: {n_hidden} n_output: {n_output}')

In [ ]:
# モデルの定義
# 784入力10出力1隠れ層のニューラルネットワークモデル

class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()

        # 隠れ層の定義 (隠れ層のノード数: n_hidden)
        self.l1 = nn.Linear(n_input, n_hidden)

        # 出力層の定義
        self.l2 = nn.Linear(n_hidden, n_output)

        # ReLU関数の定義
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.relu(x1)
        x3 = self.l2(x2)
        return x3

In [ ]:
# 訓練データセット　最初の1項目を取得
# データローダーから最初の1セットを取得する
for images, labels in train_loader:
    break

# 乱数の固定化
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

# 学習率
lr = 0.01

# モデルインスタンス生成
net = Net(n_input, n_output, n_hidden).to(device)

# 損失関数： 交差エントロピー関数
criterion = nn.CrossEntropyLoss()

# オプティマイザ: 勾配降下法
optimizer = optim.SGD(net.parameters(), lr=lr)

# 繰り返し回数
num_epochs = 10

# 評価結果記録用
history = np.zeros((0,5))

In [ ]:
# tqdmライブラリのインポート
from tqdm.notebook import tqdm

# 繰り返し計算メインループ

for epoch in range(num_epochs):
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_test = 0, 0

    # 訓練フェーズ
    for inputs, labels in tqdm(train_loader):
        n_train += len(labels)

        # GPUヘ転送
        inputs = inputs.to(device)
        labels = labels.to(device)

        #勾配の初期化
        optimizer.zero_grad()

        # 予測計算
        outputs = net(inputs)

        # 損失計算
        loss = criterion(outputs, labels)

        # 勾配計算
        loss.backward()

        # パラメータ修正
        optimizer.step()

        # 予測ラベル導出
        predicted = torch.max(outputs, 1)[1]

        # 損失と精度の計算
        train_loss += loss.item()
        train_acc += (predicted == labels).sum()

    #予測フェーズ
    for inputs_test, labels_test in test_loader:
        n_test += len(labels_test)

        inputs_test = inputs_test.to(device)
        labels_test = labels_test.to(device)


        # 予測計算
        outputs_test = net(inputs_test)

        # 損失計算
        loss_test = criterion(outputs_test, labels_test)

        #予測ラベル導出
        predicted_test = torch.max(outputs_test, 1)[1]

        # 損失と精度の計算
        val_loss +=  loss_test.item()
        val_acc +=  (predicted_test == labels_test).sum()

    # 評価値の算出・記録
    train_acc = train_acc / n_train
    val_acc = val_acc / n_test
    train_loss = train_loss * batch_size / n_train
    val_loss = val_loss * batch_size / n_test
    print (f'Epoch [{epoch+1}/{num_epochs}], loss: {train_loss:.5f} acc: {train_acc:.5f} val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}')
    item = np.array([epoch+1 , train_loss, train_acc.cpu(), val_loss, val_acc.cpu()])
    history = np.vstack((history, item))

### 結果確認

In [ ]:
#損失と精度の確認

print(f'[初期状態] loss: {history[0,3]:.5f}, accuracy: {history[0,4]:.5f}' )
print(f'[最終状態] loss: {history[-1,3]:.5f}, accuracy: {history[-1,4]:.5f}' )

In [ ]:
# 学習曲線の表示 (損失)

plt.rcParams['figure.figsize'] = (9,8)
plt.plot(history[:,0], history[:,1], 'b', label='training')
plt.plot(history[:,0], history[:,3], 'k', label='validation')
plt.xlabel('Epoc')
plt.ylabel('loss')
plt.title('Learning curve (loss)')
plt.legend()
plt.show()

In [ ]:
# 学習曲線の表示 (精度)

plt.rcParams['figure.figsize'] = (9,8)
plt.plot(history[:,0], history[:,2], 'b', label='training')
plt.plot(history[:,0], history[:,4], 'k', label='validation')
plt.xlabel('Epoc')
plt.ylabel('accuracy')
plt.title('Learning curve (accuracy)')
plt.legend()
plt.show()

**イメージ表示で確認**

In [ ]:
# DataLoaderから最初の1セットを取得する
for images, labels in test_loader:
    break

# 予測結果の取得
inputs = images.to(device)
labels = labels.to(device)
outputs = net(inputs)
predicted = torch.max(outputs, 1)[1]

In [ ]:
# 最初の50件でイメージを「正解値:予測値」と表示

plt.figure(figsize=(10, 8))
for i in range(50):
  ax = plt.subplot(5, 10, i + 1)

  # numpyに変換
  image = images[i]
  label = labels[i]
  pred = predicted[i]
  if (pred == label):
    c = 'k'
  else:
    c = 'b'

  # imgの範囲を[0, 1]に戻す
  image2 = (image + 1)/ 2

  # イメージ表示
  plt.imshow(image2.reshape(28, 28),cmap='gray_r')
  ax.set_title(f'{label}:{pred}', c=c)
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()


## 畳み込みニューラルネットワーク

畳み込みニューラルネットワーク（CNN: Convolutional Neural Network）は、画像データを扱う際によく用いられるニューラルネットワークです。

特徴として`畳み込み層`と`プーリング層`により、画素間の近傍関係を保持して学習することができます。

<img src="https://drive.google.com/uc?export=view&id=1rOoe4-ZON-SpyG4dzz1eaaiIdgpnnfaN">

### 畳み込み層

畳み込み層は、3×3や5×5などの小さな正方形領域（カーネル）を対象画像に少しずつずらしながら掛け合わせる演算をします。

<img src="https://drive.google.com/uc?export=view&id=1_d2ySG0rft6OywwDhcD3GXZ2O4ljW-4X">

畳み込みも行列演算の1つなので誤差逆伝播可能で、カーネルの形を学習します。

その結果、特定の（フィルタに似た）パターンに強く反応するフィルタを作ることができます。

PyTorchでは、レイヤー関数として[nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)が用意されています。

### プーリング層

プーリング層は、近傍（例: 3x3の範囲）の値を最大値や平均を取ることで1つにまとめる処理をします。

位置のズレが起きてもほぼ同じ出力になるため、位置ずれに頑健にする処理と言えます。

<img src="https://drive.google.com/uc?export=view&id=1sk4MdUMRozvyLqKvrJzLARGAW28PYrpm">

PyTorchでは、レイヤー関数として[nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)が用意されています。



### nn.Flatten

学習層ではありませんが、CNNを実装する際に必要になる[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)について紹介します。

`nn.Flatten`は、畳み込み処理やプーリング処理の最中は3階テンソルの形で扱われていたデータを、線形関数（全結合層）`nn.Linear`で扱えるよう、1階テンソルの形に変形します。

## 共通関数の定義

これ以降は、コードが複雑になるため、よく使う関数を共通関数として定義して使うようにします。内容には目を通して、説明できるようにしましょう。

### eval_loss 損失計算用

In [ ]:
# 損失計算用
def eval_loss(loader, device, net, criterion):

    # データローダーから最初の1セットを取得する
    for images, labels in loader:
        break

    # デバイスの割り当て
    inputs = images.to(device)
    labels = labels.to(device)

    # 予測計算
    outputs = net(inputs)

    #  損失計算
    loss = criterion(outputs, labels)

    return loss

### fit 学習用

In [ ]:
# 学習用関数
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):

    # tqdmライブラリのインポート
    from tqdm.notebook import tqdm

    base_epochs = len(history)

    for epoch in range(base_epochs, num_epochs+base_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        #訓練フェーズ
        net.train()
        count = 0

        for inputs, labels in tqdm(train_loader):
            count += len(labels)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 勾配の初期化
            optimizer.zero_grad()

            # 予測計算
            outputs = net(inputs)

            # 損失計算
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # 勾配計算
            loss.backward()

            # パラメータ修正
            optimizer.step()

            # 予測値算出
            predicted = torch.max(outputs, 1)[1]

            # 正解件数算出
            train_acc += (predicted == labels).sum()

            # 損失と精度の計算
            avg_train_loss = train_loss / count
            avg_train_acc = train_acc / count

        #予測フェーズ
        net.eval()
        count = 0

        for inputs, labels in test_loader:
            count += len(labels)
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 予測計算
            outputs = net(inputs)

            # 損失計算
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # 予測値算出
            predicted = torch.max(outputs, 1)[1]

            # 正解件数算出
            val_acc += (predicted == labels).sum()

            # 損失と精度の計算
            avg_val_loss = val_loss / count
            avg_val_acc = val_acc / count

        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
        item = np.array([epoch+1, avg_train_loss, avg_train_acc.cpu(), avg_val_loss, avg_val_acc.cpu()])
        history = np.vstack((history, item))
    return history

### eval_history 学習ログ解析用

In [ ]:
# 学習ログ解析

def evaluate_history(history):
    #損失と精度の確認
    print(f'[初期状態] loss: {history[0,3]:.5f}, accuracy: {history[0,4]:.5f}')
    print(f'[最終状態] loss: {history[-1,3]:.5f}, accuracy: {history[-1,4]:.5f}' )

    num_epochs = len(history)
    unit = num_epochs / 10

    # 学習曲線の表示 (損失)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,1], 'b', label='training')
    plt.plot(history[:,0], history[:,3], 'k', label='validation')
    plt.xticks(np.arange(0,num_epochs+1, unit))
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Learning curve (loss)')
    plt.legend()
    plt.show()

    # 学習曲線の表示 (精度)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,2], 'b', label='training')
    plt.plot(history[:,0], history[:,4], 'k', label='validation')
    plt.xticks(np.arange(0,num_epochs+1,unit))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Learning curve (accuracy)')
    plt.legend()
    plt.show()

### show_images_labels イメージとラベルの表示

In [ ]:
# イメージとラベル表示
def show_images_labels(loader, classes, net, device):

    # データローダーから最初の1セットを取得する
    for images, labels in loader:
        break
    # 表示数は50個とバッチサイズのうち小さい方
    n_size = min(len(images), 50)

    if net is not None:
      # デバイスの割り当て
      inputs = images.to(device)
      labels = labels.to(device)

      # 予測計算
      outputs = net(inputs)
      predicted = torch.max(outputs,1)[1]
      #images = images.to('cpu')

    # 最初のn_size個の表示
    plt.figure(figsize=(20, 15))
    for i in range(n_size):
        ax = plt.subplot(5, 10, i + 1)
        label_name = classes[labels[i]]
        # netがNoneでない場合は、予測結果もタイトルに表示する
        if net is not None:
          predicted_name = classes[predicted[i]]
          # 正解かどうかで色分けをする
          if label_name == predicted_name:
            c = 'k'
          else:
            c = 'b'
          ax.set_title(label_name + ':' + predicted_name, c=c, fontsize=20)
        # netがNoneの場合は、正解ラベルのみ表示
        else:
          ax.set_title(label_name, fontsize=20)
        # TensorをNumPyに変換
        image_np = images[i].numpy().copy()
        # 軸の順番変更 (channel, row, column) -> (row, column, channel)
        img = np.transpose(image_np, (1, 2, 0))
        # 値の範囲を[-1, 1] -> [0, 1]に戻す
        img = (img + 1)/2
        # 結果表示
        plt.imshow(img)
        ax.set_axis_off()
    plt.show()


### torch_seed 乱数固定用

In [ ]:
# PyTorch乱数固定用

def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True


## CIFAR-10

MNISTよりも難しいデータとしてCIFAR-10と呼ばれる機械学習用イメージ学習データを利用します。

CIFAR-10は、画素数32×32のカラーイメージデータが、airplane、automobile、birdなど10種類のカテゴリに分けられていて、イメージからカテゴリを予測する分類問題の学習データとしてよく用いられます。  
訓練用5万枚、検証用として1万枚のデータが含まれています。

<img src="https://drive.google.com/uc?export=view&id=11oJ0VYdP0NNePgBk4DS5_JC1I-tyHF7R">


## CNNの実装

In [ ]:
class CNN(nn.Module):
  def __init__(self, n_output, n_hidden):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, 3)
    self.conv2 = nn.Conv2d(32, 32, 3)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d((2,2))
    self.flatten = nn.Flatten()
    self.l1 = nn.Linear(6272, n_hidden)
    self.l2 = nn.Linear(n_hidden, n_output)

    self.features = nn.Sequential(
        self.conv1,
        self.relu,
        self.conv2,
        self.relu,
        self.maxpool)

    self.classifier = nn.Sequential(
       self.l1,
       self.relu,
       self.l2)

  def forward(self, x):
    x1 = self.features(x)
    x2 = self.flatten(x1)
    x3 = self.classifier(x2)
    return x3

In [ ]:
# Transformsの定義

# transformer2 正規化のみ実施

# 検証データ用 : 正規化のみ実施
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

# データ取得用関数 Datasets
data_root = './data'

# 訓練データセット 3階テンソル版
train_set2 = datasets.CIFAR10(
    root =  data_root, train = True,
    download = True, transform = transform2)

# 検証データセット 3階テンソル版
test_set2 = datasets.CIFAR10(
    root = data_root, train = False,
    download = True, transform = transform2)

image2, label2 = train_set2[0]

print(image2.shape)


In [ ]:
# データローダーの定義

# ミニバッチのサイズ指定
batch_size = 100


# 訓練用データローダー
# 訓練用なので、シャッフルをかける
train_loader2 = DataLoader(train_set2, batch_size=batch_size, shuffle=True)

# 検証用データローダー
# 検証時にシャッフルは不要
test_loader2 = DataLoader(test_set2,  batch_size=batch_size, shuffle=False)

# train_loader2から1セット取得
for images2, labels2 in train_loader2:
    break

# それぞれのshape確認
print(images2.shape)




In [ ]:
# 正解ラベル定義
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 検証データ最初の50個の表示
show_images_labels(test_loader2, classes, None, None)

### 学習

In [ ]:
# 乱数初期化
torch_seed()

# 入力次元数 今回は3*32*32=3072
n_input = image2.view(-1).shape[0]

# 出力次元数
# 分類先クラス数　今回は10になる
n_output = len(set(list(labels2.data.numpy())))

# 隠れ層のノード数
n_hidden = 128

# 結果確認
print(f'n_input: {n_input}  n_hidden: {n_hidden} n_output: {n_output}')

# モデルインスタンス生成
net = CNN(n_output, n_hidden).to(device)

# 損失関数： 交差エントロピー関数
criterion = nn.CrossEntropyLoss()

# 学習率
lr = 0.01

# 最適化関数: 勾配降下法
optimizer = optim.SGD(net.parameters(), lr=lr)

# 繰り返し回数
num_epochs = 10

# 評価結果記録用
history2 = np.zeros((0,5))

# 学習
history2 = fit(net, optimizer, criterion, num_epochs, train_loader2, test_loader2, device, history2)

### 評価

In [ ]:
# 評価

evaluate_history(history2)

In [ ]:
# 最初の50個の表示

show_images_labels(test_loader2, classes, net, device)

## 過学習

モデルが学習データに過度に適応してしまうことを過学習と呼びます。

パラメータ数の多い（層の数が多い、ニューロンの数が多い）ネットワークで起こりやすいとされます。

過学習の例：  
訓練での損失は減少しているが、検証では逆に損失が上昇している。

<img src="https://drive.google.com/uc?export=view&id=1uTCCwMcBalezSA3tezrP_yhV_gQ_MlSW" width=40%>

過学習を防ぐ手法として、Dropout、Data Augmentationなどが利用されます。

### Dropout

`Dropout`は、学習中に指定した層のニューロンの内ランダムに$p$%の出力を0にする処理です。$p=50$がよく用いられます。

<img src="https://drive.google.com/uc?export=view&id=1UyrqM02CVEn0pdUEUklnPX4WSfnRlZ0R" width=40%>

PyTorchでは、[nn.Dropout(0.5)](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)で提供されています。

`Dropout`は学習時のみ必要なため、推論時はオフ（すべての出力を通す）にします。

PyTorchでは、

```python
net.train()
```

で、有効化、

```python
net.eval()
```

で無効化することができます。

### Data Augmentation

Data Augmentationはデータ拡張とも呼ばれます。

学習用画像にランダムに反転，回転，切り取り，拡大縮小等を加えて学習用画像枚数を水増しします。

<img src="https://drive.google.com/uc?export=view&id=1cmjEojxAKs4BWg2W4JZLQEAWPMCE7D6u" width=40%>

モデルからみると繰り返しのたびに異なるパターンのデータが入力されるため、過学習が起きにくくなるとされます。

PyTorchでは、Transformsから簡単に利用することができます。以下のコードはランダムに左右反転する例です。

他にも[複数の種類](https://pytorch.org/vision/stable/transforms.html#scriptable-transforms)があります。


使用例
```
transform = transforms.Compose([
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.ToTensor()
])
```

## <練習問題1>

上記のCNNにDropoutを追加して、過学習が緩和されたことを確認しましょう。Dropoutは上記コードに追加しても問題ありません。
層内の自由な位置に追加して、精度がどのように変化するかを確認しましょう。

また、繰返し回数は30以上に設定してください。


例:
```
# 繰り返し回数
num_epochs = 30
```